In [1]:
import keras
from keras.preprocessing.text import Tokenizer
import string
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, SpatialDropout1D, LSTM,Input,Conv1D,GlobalMaxPooling1D,BatchNormalization,MaxPooling1D,Flatten
from keras.layers.wrappers import Bidirectional 
from keras.callbacks import ModelCheckpoint,EarlyStopping
import os
from sklearn.metrics import roc_auc_score 
import matplotlib.pyplot as plt 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.models import Model
from keras.callbacks import EarlyStopping
import re
%matplotlib inline
import numpy as np
import pandas as pd
from keras.utils import to_categorical
import h5py

c:\users\son\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### load pre processed data sets

In [2]:
fit_data = pd.read_pickle('training_set.pkl')

In [3]:
fit_data.shape

(3197922, 27)

In [4]:
fit_data.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'hour', 'min', 'day',
       'app_count', 'device_count', 'os_count', 'channel_count', 'click_count',
       'apps', 'devices', 'oss', 'channels', 'adoc', 'app_count_min',
       'device_count_min', 'os_count_min', 'channel_count_min',
       'click_count_min', 'adoc_min', 'click_count_cumsum',
       'click_count_cumsum_min', 'is_attributed'],
      dtype='object')

In [5]:
def convert_to_cat(df):
    df['device'] = df.device.astype('object')
    df['os'] = df.os.astype('object')
    df['app'] = df.app.astype('object')
    df['channel'] = df.channel.astype('object')
    df['hour'] = df.hour.astype('object')
    df['ip'] = df.ip.astype('object')
    return df

In [6]:
fit_data = convert_to_cat(fit_data)

#### convert categorical into text for word embedding

In [7]:
fit_data['cat'] = fit_data['app'].map(str)+'a '+fit_data['channel'].map(str)+'c '+ fit_data['device'].map(str)+'d '+\
fit_data['os'].map(str)+['os ']

In [8]:
fit_data.head(1)

,ip,app,device,os,channel,hour,min,day,app_count,device_count,...,app_count_min,device_count_min,os_count_min,channel_count_min,click_count_min,adoc_min,click_count_cumsum,click_count_cumsum_min,is_attributed,cat
975869,109434,2,1,19,477,16,21,6,1.386294,0.0,...,0.0,0.0,0.0,0.0,0.693147,5.814131,10,0,0,2a 477c 1d 19os


In [9]:
# output directory name:
output_dir = 'model_output/parallel/t50/'
n_unique_words = 10000 
max_text_length = 4 
pad_type = trunc_type = 'pre'

In [10]:
# instantiate tokenizer
t = Tokenizer(num_words = n_unique_words)
# index words on corpus
t.fit_on_texts(fit_data.cat.astype(str))

#### save tokenizer to use on test data later

In [46]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(t, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
fit_data['token_vec'] = t.texts_to_sequences(fit_data.cat.astype(str))

In [12]:
def pad_seq(sequence):
    return pad_sequences(sequence,maxlen=max_text_length,dtype='int32',padding='pre',truncating='pre',value=0.)

In [13]:
train,validation = train_test_split(fit_data,test_size=0.3, random_state = 10)

In [14]:
print(train.size,validation.size)

64917805 27821933


In [15]:
validation[validation.is_attributed==1].shape[0]/validation.shape[0]

0.14325129745657858

In [16]:
train[train.is_attributed==1].shape[0]/train.shape[0]

0.14268821935676967

In [17]:
df1 = train[train.is_attributed==1].copy()

In [18]:
frames = [train,df1]
train = pd.concat(frames)

In [19]:
train[train.is_attributed==1].shape[0]/train.shape[0]

0.24974129765176065

In [20]:
train =train.sample(frac=1.0)

In [21]:
del fit_data

#### categorical features

In [22]:
train_cat = pad_seq(train.token_vec)
validation_cat = pad_seq(validation.token_vec)

In [23]:
train.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'hour', 'min', 'day',
       'app_count', 'device_count', 'os_count', 'channel_count', 'click_count',
       'apps', 'devices', 'oss', 'channels', 'adoc', 'app_count_min',
       'device_count_min', 'os_count_min', 'channel_count_min',
       'click_count_min', 'adoc_min', 'click_count_cumsum',
       'click_count_cumsum_min', 'is_attributed', 'cat', 'token_vec'],
      dtype='object')

#### labels

In [24]:
label_train = train.is_attributed.values
label_valid = validation.is_attributed.values

#### numeric features

In [25]:
train_feat = train[['ip','app_count','device_count','os_count','channel_count','click_count','apps',
                 'devices','oss','channels','adoc','app_count_min','device_count_min','os_count_min',
                  'channel_count_min','click_count_min','adoc_min','click_count_cumsum','click_count_cumsum_min'
                 ]].values
validation_feat =validation[['ip','app_count','device_count','os_count','channel_count','click_count','apps',
                 'devices','oss','channels','adoc','app_count_min','device_count_min','os_count_min',
                  'channel_count_min','click_count_min','adoc_min','click_count_cumsum','click_count_cumsum_min'
                 ]].values

In [26]:
del train
del validation

In [27]:
print(train_feat.shape,validation_feat.shape)

(2557959, 19) (959377, 19)


In [28]:
print(train_cat.shape,validation_cat.shape)

(2557959, 4) (959377, 4)


#### nn design

In [29]:
input_cat = Input(shape = (max_text_length,), name = 'input_cat')
# cat ------
cat = Embedding(n_unique_words,4)(input_cat)
cat = SpatialDropout1D(0.1)(cat)
cat = Conv1D(16, 2, activation='relu')(cat)
#cat = Conv1D(128, 3, activation='relu')(cat)
#cat = Conv1D(256, 3, activation='relu')(cat)
#cat = Conv1D(512, 2, activation='relu')(cat)
cat = GlobalMaxPooling1D()(cat)
#output_cat = Dense(2,activation='sigmoid',name='output_cat')(cat)
# num --------------
input_num = Input(shape = (19,),name='input_num')
num = BatchNormalization()(input_num)
num = Dense(64, activation='relu')(num)
#num = BatchNormalization()(num)
#num = Dense(128, activation='relu')(num)
#num = Dense(256, activation='relu')(num)
num = Dropout(0.1)(num)
#output_num = Dense(2,activation='sigmoid',name='output_num')(num)
# comb ------------
x = keras.layers.concatenate([cat, num])
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
#x = Dense(256, activation='relu')(x)
#x = Dense(512, activation='relu')(x)
#x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
output_main = Dense(1,activation='sigmoid',name='output_main')(x)

In [30]:
model = Model(inputs=[input_cat, input_num], outputs=[output_main])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_cat (InputLayer)          (None, 4)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 4, 4)         40000       input_cat[0][0]                  
__________________________________________________________________________________________________
input_num (InputLayer)          (None, 19)           0                                            
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 4, 4)         0           embedding_1[0][0]                
__________________________________________________________________________________________________
batch_norm

In [31]:
model.compile(#loss='categorical_crossentropy',
               loss = 'binary_crossentropy',
               #optimizer='RMSprop',
               optimizer='adam',
              #optimizer = sgd,
              metrics=['accuracy'])

In [32]:
output_dir

'model_output/parallel/t50/'

In [33]:
earlystop = EarlyStopping(patience=10)
modelsave = ModelCheckpoint(
    filepath=output_dir+"best_model.hdf5", save_best_only=True, verbose=1)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [34]:
history=model.fit({'input_cat':train_cat, 'input_num':train_feat},
                  {'output_main':label_train},
                  batch_size=4096*8,
                  epochs=100,
                  verbose=1,
                  validation_split=0.2,
                  callbacks=[earlystop, modelsave]
                 )

Train on 2046367 samples, validate on 511592 samples
Epoch 1/100
2046367/2046367 [==============================] - 105s 51us/step - loss: 0.2771 - acc: 0.9073 - val_loss: 0.1944 - val_acc: 0.9458

Epoch 00001: val_loss improved from inf to 0.19445, saving model to model_output/parallel/t50/best_model.hdf5
Epoch 2/100
2046367/2046367 [==============================] - 100s 49us/step - loss: 0.1774 - acc: 0.9468 - val_loss: 0.1617 - val_acc: 0.9511

Epoch 00002: val_loss improved from 0.19445 to 0.16165, saving model to model_output/parallel/t50/best_model.hdf5
Epoch 3/100
2046367/2046367 [==============================] - 100s 49us/step - loss: 0.1658 - acc: 0.9491 - val_loss: 0.1596 - val_acc: 0.9511

Epoch 00003: val_loss improved from 0.16165 to 0.15958, saving model to model_output/parallel/t50/best_model.hdf5
Epoch 4/100
2046367/2046367 [==============================] - 100s 49us/step - loss: 0.1602 - acc: 0.9504 - val_loss: 0.1568 - val_acc: 0.9513

Epoch 00004: val_loss improve


Epoch 00036: val_loss improved from 0.14367 to 0.14367, saving model to model_output/parallel/t50/best_model.hdf5
Epoch 37/100
2046367/2046367 [==============================] - 117s 57us/step - loss: 0.1442 - acc: 0.9534 - val_loss: 0.1437 - val_acc: 0.9535

Epoch 00037: val_loss did not improve
Epoch 38/100
2046367/2046367 [==============================] - 105s 52us/step - loss: 0.1441 - acc: 0.9535 - val_loss: 0.1446 - val_acc: 0.9534

Epoch 00038: val_loss did not improve
Epoch 39/100
2046367/2046367 [==============================] - 117s 57us/step - loss: 0.1441 - acc: 0.9535 - val_loss: 0.1445 - val_acc: 0.9535

Epoch 00039: val_loss did not improve
Epoch 40/100
2046367/2046367 [==============================] - 106s 52us/step - loss: 0.1440 - acc: 0.9535 - val_loss: 0.1441 - val_acc: 0.9536

Epoch 00040: val_loss did not improve
Epoch 41/100
2046367/2046367 [==============================] - 119s 58us/step - loss: 0.1439 - acc: 0.9535 - val_loss: 0.1435 - val_acc: 0.9536

Epo

2046367/2046367 [==============================] - 105s 51us/step - loss: 0.1404 - acc: 0.9543 - val_loss: 0.1403 - val_acc: 0.9546

Epoch 00076: val_loss improved from 0.14047 to 0.14032, saving model to model_output/parallel/t50/best_model.hdf5
Epoch 77/100
2046367/2046367 [==============================] - 100s 49us/step - loss: 0.1404 - acc: 0.9543 - val_loss: 0.1405 - val_acc: 0.9546

Epoch 00077: val_loss did not improve
Epoch 78/100
2046367/2046367 [==============================] - 100s 49us/step - loss: 0.1404 - acc: 0.9543 - val_loss: 0.1404 - val_acc: 0.9546

Epoch 00078: val_loss did not improve
Epoch 79/100
2046367/2046367 [==============================] - 102s 50us/step - loss: 0.1402 - acc: 0.9544 - val_loss: 0.1409 - val_acc: 0.9544

Epoch 00079: val_loss did not improve
Epoch 80/100
2046367/2046367 [==============================] - 101s 49us/step - loss: 0.1403 - acc: 0.9544 - val_loss: 0.1405 - val_acc: 0.9544

Epoch 00080: val_loss did not improve
Epoch 81/100
2046

In [45]:
np.save("./train_cat", train_cat,allow_pickle=True)
np.save("./train_feat", train_feat,allow_pickle=True)
np.save("./validation_cat", validation_cat,allow_pickle=True)
np.save("./validation_feat", validation_feat,allow_pickle=True)
np.save("./label_train", label_train,allow_pickle=True)
np.save("./label_valid", label_valid,allow_pickle=True)

#### evaluate on validation set

In [35]:
from keras.models import load_model
model=load_model(output_dir+"best_model.hdf5")

In [36]:
y_p = model.predict({'input_cat':validation_cat, 'input_num':validation_feat})

In [93]:
#y_p = np.argmax(y_p, axis=1)

In [37]:
output_dir

'model_output/parallel/t50/'

In [40]:
y_true = label_valid

In [41]:
from sklearn.metrics import roc_auc_score
auc_train = roc_auc_score(y_true, y_p)

In [97]:
auc_train

0.97274782732077569

In [42]:
auc_train # train4

0.9749821339421685

#### testing set 

In [100]:
100*np.argmax(y,axis=1).sum()/len(y)

0.0

In [101]:
#y_c = np.argmax(y, axis=1)
y = pd.DataFrame(y)
y.columns = ['is_attributed']
y.head()

,is_attributed
0,0.013694
1,0.011774
2,0.004334
3,0.003711
4,0.003187


In [102]:
#y_c = np.argmax(y, axis=1)
#y = pd.DataFrame(y_c)
#y.columns = ['is_attributed']
#y.head()

In [103]:
y.shape

(18790469, 1)

In [104]:
results = test[['click_id']].join(y)
results.head()

,click_id,is_attributed
0,0,0.013694
1,1,0.011774
2,2,0.004334
3,3,0.003711
4,4,0.003187


In [105]:
results.to_csv(output_dir+'results.csv',index=False)

In [106]:
output_dir

'model_output/parallel/t50/'